In [47]:
import pandas as pd
import planet
import asyncio
import os
import getpass
from planet import Auth, Session
import json

DOWNLOAD_DIR = '../data/planet_data'


In [2]:
user = input("Username: ")
pw = getpass.getpass()
auth = Auth.from_login(user,pw)
auth.store()

Interaction with the server is done through sessions that automatically implement rate limits.
Do not use multiple Sessions as it can cause collisions and bypass rate-limiting.

Searching:

In [ ]:
def parse_polygon(polygon_str):
    # Remove the 'POLYGON ((' prefix and '))' suffix
    polygon_str = polygon_str.replace('POLYGON ((', '').replace('))', '')
    # Split the string into coordinate pairs
    coord_pairs = polygon_str.split(', ')
    # Convert each coordinate pair into a list of floats
    coordinates = [[float(coord) for coord in pair.split()] for pair in coord_pairs]
    # Wrap the list of coordinates in an additional list to match the desired structure
    return [coordinates]

data = pd.read_csv("../data/places.csv")

data['geometry'] = data['geometry'].apply(parse_polygon)

geom = {
       "type": "Polygon",
       "coordinates": data['geometry'][0]
   }

In [26]:
from planet import data_filter
from datetime import datetime
# Define the filters we'll use to find our data

item_types = ["PSScene"]

# set individual filters

# geom filter: search for items whose footprint geometry inter-
# sects with specified geometry
geom_filter = data_filter.geometry_filter(geom)

# # only include images with clear_percent greater than 90 
# clear_percent_filter = data_filter.range_filter('clear_percent', gt = 90)

# only look for images between following dates
date_range_filter = data_filter.date_range_filter("acquired", gt = datetime(month=1, day=1, year=2022), lt=datetime(month=1, day=3, year=2022))

# # cloudcover less than 0.1
# cloud_cover_filter = data_filter.range_filter('cloud_cover', lt = 0.1)

# combine individual filters
combined_filter = data_filter.and_filter([geom_filter, date_range_filter])

In [30]:
async with Session() as sess:
    cl = sess.client('data')
    request = await cl.create_search(name='planet_client_demo',
                                     search_filter=combined_filter,
                                     item_types=item_types)

In [31]:
# The limit paramter allows us to limit the number of results from our search that are returned.
# The default limit is 100. Here, we're setting our result limit to 50.
async with Session() as sess:
    cl = sess.client('data')
    items = cl.run_search(search_id=request['id'], limit=50)
    item_list = [i async for i in items]

In [ ]:
for item in item_list:
    print(item['id'], item['properties']['item_type'])

# and save results
with open('../search_results/results.json','w') as f:
    jsonStr = json.dumps(item_list)
    f.write(jsonStr)
    f.close()

In [ ]:
item_list[0]["properties"]

Creating Orders:

In [ ]:
def create_request(geom, items, request_name, item_type, asset_type):
   '''
   This function creates an order request for the specified geometry, items, request name, item type, and asset type.
   geom: The geometry of the location of interest. This is used to clip the images to the specified area.
   items: The list of item IDs to be ordered.
   request_name: The name of the order request (for cataloging).
   item_type: The type of item to be ordered, e.g. PSScene
   asset_type: The type of asset to be ordered, e.g. ortho_visual, ortho_analytic_4b 
   '''
   order = planet.order_request.build_request(
       name=request_name,
       products=[
           planet.order_request.product(item_ids=items,
                                        product_bundle=asset_type,
                                        item_type=item_type)
       ],
       tools=[planet.order_request.clip_tool(aoi=geom)])

   return order

In [48]:
async def create_and_download(client, order_detail, directory):
   with planet.reporting.StateBar(state='creating') as reporter:
       order = await client.create_order(order_detail)
       reporter.update(state='created', order_id=order['id'])
       await client.wait(order['id'], callback=reporter.update_state)

   await client.download_order(order['id'], directory, progress_bar=True)

async def main(geom, items, request_name, item_type, asset_type):
   async with planet.Session() as sess:
       cl = sess.client('orders')

       # Create the order request
       request = create_request(geom, items, request_name, item_type, asset_type)

       # Create and download the order
       order = await create_and_download(cl, request, DOWNLOAD_DIR)

In [ ]:
await main(geom, [item['id'] for item in item_list], 'planet_client_demo', 'PSScene', 'visual')